after running the verification experiment tutorial_global_oce_latlon in "myrun" with useSingleCPUio = .TRUE., dumpInitAndLast = .FALSE., monitorFreq = 2592000., dumpFreq = 0., taveFreq = 311040000., ( pChkptFreq= 311040000. to reduce expensive output), for 100 years, run this notebook to plot some fields

In [26]:
%matplotlib widget


import numpy as np
import matplotlib.pyplot as plt
import os
from xmitgcm import open_mdsdataset
# helper function
def sq(a):
    a = np.squeeze(a)
    return np.ma.masked_array(a==0.,a)


bdir = '/Users/mlosch/Documents/teaching/GLOMAR/AdvancedOceanModelling/MITgcm/verification'
experiment = 'tutorial_global_oce_latlon/myrun'

In [27]:
import warnings
warnings.filterwarnings('ignore')
# suppress numerous warnings only when reading data into the workspace with xmitgcm.open_mdsdataset
with warnings.catch_warnings():
    # this will suppress all warnings in this block
    warnings.simplefilter("ignore")
    ds = open_mdsdataset(os.path.join(bdir,experiment), 
                         prefix = ['Eta2tave', 'wVeltave', 'sFluxtave', 'tFluxtave', 'PTRtave01', 
                                   'Convtave', 'VTtave', 'vFluxtave', 'uFluxtave', 'PhHytave', 
                                   'PHLtave', 'VStave', 'Ttave', 'VVtave', 'UTtave', 'GM_Kwy-T', 'GM_Kwx-T', 
                                   'PHL2tave', 'UStave', 'WTtave', 'vVeltave', 'GM_Kwz-T', 'UVtave', 'Tdiftave',
                                   'ETAtave', 'UUtave', 'TTtave', 'WStave', 'Stave', 'uVeltave'], 
                                    delta_t=86400., ref_date="2001-1-1 0:0:0", geometry='sphericalpolar')

In [28]:
plt.close('all')
fig1,ax1 = plt.subplots(2,2,sharex=True,sharey=True,figsize=(11,6))
# replace zeros by NaN
theta=ds.Ttave.where(ds.Ttave!=0)
salt=ds.Stave.where(ds.Stave!=0)
# plot last step
k = -1
ph0 = theta.isel(time=k,Z=0).plot(ax=ax1[0,0],vmin=-2, vmax = 30)
ph1 = salt.isel(time=k,Z=0).plot(ax=ax1[0,1])
ph2 = theta.isel(time=k,Z=10).plot(ax=ax1[1,0],vmin=-2, vmax = 4)
ph3 = salt.isel(time=k,Z=10).plot(ax=ax1[1,1])
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

compute stream horizontal stream function and Eulerian overturning stream function 

**Note: Eulerian means is not the correct way of computing the overturning stream function, we do it here for simplicity only!!**

In [29]:
k=-1
eta = ds.ETAtave.isel(time=k).where(ds.Depth!=0)

uh = (ds.uVeltave.isel(time=k) * ds.hFacW * ds.drF * ds.dyG).sum(axis=0)
# replace zeros by NaN
# by convention, I integrate from north to south; masking is only approximate
barostrfct = (uh[::-1,:].cumsum(axis=0)[::-1,:]).where(uh!=0)*1e-6

# Eulerian average is not the correct way of doing this, but it's simple, so we do it for illustration only!!!
vv = (ds.vVeltave.isel(time=k) * ds.hFacS * ds.drF * ds.dxG).sum(axis=-1)
# integrate from top to bottom for Eulerian
ovrtstrfct = -vv[::-1,:].cumsum(axis=0).where(vv!=0) * 1e-6

fig2,ax2 = plt.subplots(3,1,figsize=(10,12))
phe = eta.plot.contourf(ax=ax2[0],cmap=plt.cm.viridis,levels=20)
phb = barostrfct.plot.contourf(ax=ax2[1],cmap=plt.cm.viridis,levels=20)
pho = ovrtstrfct.plot.contourf(ax=ax2[2],cmap=plt.cm.viridis,levels=20)
ax2[1].title.set_text('Barotropic streamfunction (Sv)')
ax2[2].title.set_text('Eulerian Overturning streamfunction (Sv) (DON''T USE)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …